# Two task network 

Network has eight inputs:

1. The fixation. 
1. $u_{rule}^{1}$
1. $u_{rule}^{2}$
1. The first context mod. 
1. The second ontext mod. 
1. The first context status. 
1. The second context status. 
1. The Romo signals.

Network has five outputs: 
1. The fixation. 
1. The first context output. 
1. The second context output. 
1. The first Romo task output. 
1. The second Romo task output. 


<div>
<img src="./images/Sheme.png" width="300"/>
</div>

> Learning rule: superspike

> Neuron type: Lif


$$\begin{align*}
            \dot{v} &= 1/\tau_{\text{mem}} (v_{\text{leak}} - v + i) \\
            \dot{i} &= -1/\tau_{\text{syn}} i
        \end{align*}
$$ 

In [1]:
import torch
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt  # for analys
from cgtasknet.net.lif import SNNLif
from cgtasknet.tasks.tasks import MultyTask
from norse.torch.functional.lif import LIFParameters

## Step -1: Select device

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f'Device: {("gpu (cuda)" if device.type=="cuda" else "cpu")}')

Device: gpu (cuda)


In [3]:
batch_size = 20
number_of_tasks = 10
task_list = [("WorkingMemory", dict()), (("ContextDM", dict()))]
tasks = dict(task_list)
Task = MultyTask(tasks=tasks, batch_size=15)

## Step 1.1: Create model

In [4]:
feature_size, output_size = Task.feature_and_act_size[0]
hidden_size = 600

neuron_parameters = LIFParameters()
model = SNNLif(
    feature_size, hidden_size, output_size, neuron_parameters=neuron_parameters
).to(device)

## Step 1.2: Save pre-learning weights

In [5]:
weights_pre_l = []
with torch.no_grad():
    for name, param in model.named_parameters():
        if param.requires_grad:
            weights_pre_l.append((param).cpu().numpy())

## Step 2: loss and creterion 

In [6]:
learning_rate = 5e-2
criterion = nn.MSELoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.8, 0.85))
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

## Step 3: Train loop

In [7]:
%matplotlib
plt.ion
fig = plt.figure()
ax = fig.add_subplot(111)
fig2 = plt.figure()
ax2 = fig2.add_subplot(111)
inputs, target_outputs = Task.dataset(number_of_tasks)
(line1,) = ax.plot(np.arange(0, len(target_outputs)), target_outputs[:, 0, 1], "b-")
(line2,) = ax.plot(np.arange(0, len(target_outputs)), target_outputs[:, 0, 2], "r-")
(line3,) = ax.plot(np.arange(0, len(target_outputs)), target_outputs[:, 0, 1], "b-")
(line4,) = ax.plot(np.arange(0, len(target_outputs)), target_outputs[:, 0, 2], "r-")
(line21,) = ax2.plot(np.arange(0, len(target_outputs)), target_outputs[:, 0, 1], "b-")
(line22,) = ax2.plot(np.arange(0, len(target_outputs)), target_outputs[:, 0, 2], "r-")
(line23,) = ax2.plot(np.arange(0, len(target_outputs)), target_outputs[:, 0, 1], "b-")
(line24,) = ax2.plot(np.arange(0, len(target_outputs)), target_outputs[:, 0, 2], "r-")
ax.set_ylim([-0.5, 1.5])
ax.set_xlim([0, 20000])
ax2.set_ylim([-0.5, 1.5])
ax2.set_xlim([0, 20000])
running_loss = 0
fig.canvas.draw()
fig.canvas.flush_events()
fig2.canvas.draw()
fig2.canvas.flush_events()
ax.set_title("LIF")
ax2.set_title("LIF")
for i in range(2000):
    inputs, target_outputs = Task.dataset(number_of_tasks)
    inputs += np.random.normal(0, 0.01, size=(inputs.shape))
    inputs = torch.from_numpy(inputs).type(torch.float).to(device)
    target_outputs = torch.from_numpy(target_outputs).type(torch.float).to(device)

    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs, states = model(inputs)

    loss = criterion(outputs, target_outputs)
    loss.backward()
    optimizer.step()

    # print statistics
    running_loss += loss.item()
    if i % 10 == 9:
        print("epoch: {:d} loss: {:0.5f}".format(i + 1, running_loss / 10))
        running_loss = 0.0
        with torch.no_grad():
            inputs, target_outputs = Task.dataset(number_of_tasks)

            inputs = torch.from_numpy(inputs).type(torch.float).to(device)
            target_outputs = (
                torch.from_numpy(target_outputs).type(torch.float).to(device)
            )
            outputs, states = model(inputs)
            loss = criterion(outputs, target_outputs)

            print("test loss: {:0.5f}".format(loss.item()))
        for_plot = outputs.detach().cpu().numpy()[:, 0, :]
        print(len(for_plot))
        line1.set_xdata(np.arange(0, len(for_plot), 1))
        line2.set_xdata(np.arange(0, len(for_plot), 1))
        line3.set_xdata(np.arange(0, len(for_plot), 1))
        line4.set_xdata(np.arange(0, len(for_plot), 1))
        line21.set_xdata(np.arange(0, len(for_plot), 1))
        line22.set_xdata(np.arange(0, len(for_plot), 1))
        line23.set_xdata(np.arange(0, len(for_plot), 1))
        line24.set_xdata(np.arange(0, len(for_plot), 1))

        line1.set_ydata(for_plot[:, 1])
        line2.set_ydata(for_plot[:, 2])
        line3.set_ydata(target_outputs.detach().cpu().numpy()[:, 0, 1])
        line4.set_ydata(target_outputs.detach().cpu().numpy()[:, 0, 2])

        line21.set_ydata(for_plot[:, 3])
        line22.set_ydata(for_plot[:, 4])
        line23.set_ydata(target_outputs.detach().cpu().numpy()[:, 0, 3])
        line24.set_ydata(target_outputs.detach().cpu().numpy()[:, 0, 4])

    fig.canvas.draw()
    fig.canvas.flush_events()
    fig2.canvas.draw()
    fig2.canvas.flush_events()


print("Finished Training")

Using matplotlib backend: TkAgg
epoch: 10 loss: 0.14683
test loss: 0.12054
22394
epoch: 20 loss: 0.11673
test loss: 0.11673
21793
epoch: 30 loss: 0.11003
test loss: 0.10265
23596
epoch: 40 loss: 0.10544
test loss: 0.10650
22394
epoch: 50 loss: 0.10133
test loss: 0.10294
22394
epoch: 60 loss: 0.09789
test loss: 0.09120
24197
epoch: 70 loss: 0.09374
test loss: 0.10342
21192
epoch: 80 loss: 0.09365
test loss: 0.09090
22995
epoch: 90 loss: 0.09124
test loss: 0.08994
22394
epoch: 100 loss: 0.08734
test loss: 0.08944
22394
epoch: 110 loss: 0.08899
test loss: 0.08247
23596
epoch: 120 loss: 0.08387
test loss: 0.08466
22394
epoch: 130 loss: 0.08437
test loss: 0.08162
22995
epoch: 140 loss: 0.08068
test loss: 0.07964
22995
epoch: 150 loss: 0.07827
test loss: 0.07932
22394
epoch: 160 loss: 0.07830
test loss: 0.07175
24197
epoch: 170 loss: 0.07751
test loss: 0.07310
23596
epoch: 180 loss: 0.07272
test loss: 0.06667
24798
epoch: 190 loss: 0.07471
test loss: 0.07011
23596
epoch: 200 loss: 0.07651
te

KeyboardInterrupt: 